# Assignment 3 (Memory)  - Ollama (Gemma2 결과)


In [13]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.chat_models.ollama import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import (
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate,
)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.memory import ConversationBufferMemory


# 챗 지피티
chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 로컬 LLM (ollama)
chat_ollama = ChatOllama(
    temperature=0.1,
    # model="mixtral:instruct",
    # model="stablelm2:latest",
    # model="gemma2:2b",
    model="gemma2:latest",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [
    {"movie": "탑건", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "쥬라기 공원", "answer": "🦖🏞️🚗"},
    {"movie": "해리 포터", "answer": "🧙‍♂️⚡🏰"},
    {"movie": "토이 스토리", "answer": "🤠🚀🧸"},
    {"movie": "타이타닉", "answer": "🚢💏🧊"},
    {"movie": "인셉션", "answer": "💤🌀🕰️"},
    {"movie": "어벤져스", "answer": "🦸‍♂️🦸‍♀️🌌"},
    {"movie": "라이온 킹", "answer": "🦁👑🌅"},
    {"movie": "겨울왕국", "answer": "❄️👸🎶"},
    {"movie": "매트릭스", "answer": "🕶️💊🐇"},
    {"movie": "스타워즈", "answer": "⭐🚀🤖"},
]


memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)

# memory = ConversationSummaryBufferMemory(
#     llm=chat,
#     max_token_limit=120,
#     memory_key="history",
#     return_messages=True,
# )


def load_memory(_):
    return memory.load_memory_variables({})["history"]


output_parser = StrOutputParser()

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 제목을 말하면 영화를 설명하는 이모티콘으로 답변해줘. 영화제목은 {movie}야",
        ),
        ("ai", "{movie} : {answer}\n"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)


explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 대해 이모티콘 3개로 설명할 수 있어."),
        example_prompt,
        ("human", "{movie}"),
    ]
)

find_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 이제까지의 대화내용을 보고 나에게 말해 줄 수 있어."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 체인 생성 (중간에 chat_ollama를 넣어서 로컬 LLM을 사용함 -> chat으로 바꾸면 오픈AI 사용)
explain_chain = explain_prompt | chat_ollama | output_parser

find_chain = (
    RunnablePassthrough.assign(history=load_memory)
    | find_prompt
    | chat_ollama
    | output_parser
)


def invoke_chain(question):
    result = explain_chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print("")

In [14]:
invoke_chain("로보캅")

invoke_chain("석양의 무법자")

invoke_chain("핑크팬더")

로보캅 : 🤖👮💥 

석양의 무법자 : 🤠🐎🏜️ 

핑크팬더 : 🐼💖🎂 



In [15]:
find_chain.invoke({"question": "내가 첫번째에 물어본 영화제목은 뭐야?"})
find_chain.invoke({"question": "내가 두번째에 물어본 영화제목은 뭐야?"})
find_chain.invoke({"question": "내가 세번째에 물어본 영화제목은 뭐야?"})
memory.load_memory_variables({})["history"]

네, 이제까지의 대화 내용을 기억하고 있습니다. 

당신이 첫 번째로 물어본 영화 제목은 **로보캅** 이었습니다. 🤖👮💥  


두 번째에 물어본 영화 제목은 **석양의 무법자** 입니다. 🤠🐎🏜️  
네, 세 번째로 물어본 영화 제목은 **핑크팬더** 입니다.  🐼💖🎂  


[HumanMessage(content='로보캅'),
 AIMessage(content='로보캅 : 🤖👮💥 \n'),
 HumanMessage(content='석양의 무법자'),
 AIMessage(content='석양의 무법자 : 🤠🐎🏜️ \n'),
 HumanMessage(content='핑크팬더'),
 AIMessage(content='핑크팬더 : 🐼💖🎂 \n')]

# Assignment 3 (Memory)  - GPT4o-mini

In [16]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.chat_models.ollama import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import (
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate,
)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.memory import ConversationBufferMemory


# 챗 지피티
chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 로컬 LLM (ollama)
chat_ollama = ChatOllama(
    temperature=0.1,
    # model="mixtral:instruct",
    # model="stablelm2:latest",
    # model="gemma2:2b",
    model="gemma2:latest",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [
    {"movie": "탑건", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "쥬라기 공원", "answer": "🦖🏞️🚗"},
    {"movie": "해리 포터", "answer": "🧙‍♂️⚡🏰"},
    {"movie": "토이 스토리", "answer": "🤠🚀🧸"},
    {"movie": "타이타닉", "answer": "🚢💏🧊"},
    {"movie": "인셉션", "answer": "💤🌀🕰️"},
    {"movie": "어벤져스", "answer": "🦸‍♂️🦸‍♀️🌌"},
    {"movie": "라이온 킹", "answer": "🦁👑🌅"},
    {"movie": "겨울왕국", "answer": "❄️👸🎶"},
    {"movie": "매트릭스", "answer": "🕶️💊🐇"},
    {"movie": "스타워즈", "answer": "⭐🚀🤖"},
]


memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)

# memory = ConversationSummaryBufferMemory(
#     llm=chat,
#     max_token_limit=120,
#     memory_key="history",
#     return_messages=True,
# )


def load_memory(_):
    return memory.load_memory_variables({})["history"]


output_parser = StrOutputParser()

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 제목을 말하면 영화를 설명하는 이모티콘으로 답변해줘. 영화제목은 {movie}야",
        ),
        ("ai", "{movie} : {answer}\n"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)


explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 대해 이모티콘 3개로 설명할 수 있어."),
        example_prompt,
        ("human", "{movie}"),
    ]
)

find_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 이제까지의 대화내용을 보고 나에게 말해 줄 수 있어."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 체인 생성 (중간에 chat_ollama를 넣어서 로컬 LLM을 사용함 -> chat으로 바꾸면 오픈AI 사용)
explain_chain = explain_prompt | chat | output_parser

find_chain = (
    RunnablePassthrough.assign(history=load_memory) | find_prompt | chat | output_parser
)


def invoke_chain(question):
    result = explain_chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print("")

In [17]:
invoke_chain("로보캅")

invoke_chain("석양의 무법자")

invoke_chain("핑크팬더")

로보캅 : 🤖👮‍♂️🔫
석양의 무법자 : 🌅🔫🤠
핑크팬더 : 🐾💖🕵️‍♂️


In [18]:
find_chain.invoke({"question": "내가 첫번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 두번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 세번째에 물어본 영화제목은 뭐야?"})
print("")
memory.load_memory_variables({})["history"]

당신이 첫 번째에 물어본 영화 제목은 "로보캅"입니다.두 번째로 물어본 영화 제목은 "석양의 무법자"입니다.당신이 세 번째에 물어본 영화 제목은 "핑크팬더"입니다.

[HumanMessage(content='로보캅'),
 AIMessage(content='로보캅 : 🤖👮\u200d♂️🔫'),
 HumanMessage(content='석양의 무법자'),
 AIMessage(content='석양의 무법자 : 🌅🔫🤠'),
 HumanMessage(content='핑크팬더'),
 AIMessage(content='핑크팬더 : 🐾💖🕵️\u200d♂️')]